# Validating Peak to Target Gene Predictions With Hi-C Data

## Critical issues with using ChIP-seq nearest gene TSS as a ground truth datset for evaluating GRN inference results

ChIP-seq datasets provide transcription factor (TF) to peak binding information, but they do not provide any information about potential enhancer (ATAC-seq peaks) to target gene (TG) regulatory interactions. A common approach when using ChIP-seq data as a validation set for TF to TG regulatory interactions (TF-TG) is to set the TG for the measured TF-peak interactions as the gene with the closest transcription start site (TSS) to the peak. However, this approach ignores long-range and multiple peak to TG interactions, which have been extensively shown to make up a large proportion - if not a majority - of enhancer-TG interactions<sup>1-14</sup>.

Most peak to TG regulatory potential inference tools attempt to model both long-range and multiple peak-TG regulatory interactions. If the validation set uses nearest gene TSS to link peaks to TGs, then any peak-TG prediction other than that between the peak and the closest gene will evaluate as false according to the validation set - regardless of biological truth.

If the validation sets used to train and evaluate predictions from a machine learning model contain a high proportion of biologically inaccurate TF-TG interactions, then any predictive model which is accurate compared to the validation set will be biologically inaccurate. 

Suppose that a gene regualtory network (GRN) inference method correctly predicts every biologically true TF-TG interaction and every biologically false TF-TG interaction. If the accuracy of these predictions were tested against a validation set containing inaccurate TF-TG interactions, then the biologically accurate predictions would be evaluated as having a low accuracy compared to the validation set.

## Potential Solutions

In light of these issues with building validation sets using the nearest TSS method, other validation methods must be used to evalutate the accuracy of peak-TG regualtory potential predictions. 

### Experimental Validation of TF-peak Binding

While ChIP-seq cannot provide any information about TF-TG binding alone, it still provides valuable information about the locations for TF-peak binding.

### Experimental Validation of peak-TG Regulation

Hi-C data identifies locations on the genome which physically interact. This can help to validate peak-TG regulatory potential, as enhancers physically interact with the genes they regulate during transcription. We can compare peak-TG predictions on whether the region containing the peak is known to interact with the region containing the gene TSS. For regions containing more than one potential target, we can either:

1. Set a potential binding score for each gene in the region, calculated as the probablility of the peak regulating a gene in the region rather than any other gene in the region. For example, if the Hi-C pair covers 10 genes, then the probability that the peak regulates any given gene in the region would be 1/10.

2. Use the closest gene target within the Hi-C pair.

3. Add a peak-TG pair for each gene in the region of overlap.

### Creating a TF-TG Validation Set

We can combine the ChIP-seq TF-peak scores with the peak-TG Hi-C chromatin interaction data by peak to create a ground truth which captures long-range and multiple peak-TG regualtory interactions.


---

## Creating a Hi-C peak-TG Validation Set for mESC

I downloaded an [in situ Hi-C dataset for R1 mESC cells](https://data.4dnucleome.org/experiment-set-replicates/4DNESMXBLGKA/) from the [4D Nucleosome Data Portal](https://data.4dnucleome.org/). We will use the [Cooler](https://github.com/open2c/cooler) Python package to work with the Hi-C dataset.

The script `dev/testing_scripts/map_peaks_to_hic.py` uses the Hi-C dataset to find chromatin contact frequencies between each gene's TSS from an scRNA-seq parsed `peak_df.parquet` file and all peaks within 1 Mb on the same chromosome This is saved as a sparse matrix `.npz` file of non-zero peak-gene contact values. Extracting the contact values for each peak-gene pair is resource intensive but highly parallelizable, so I also created the `dev/testing_scripts/run_map_peaks_to_hic.sh` script to execute the job on the HPC.

I located all peak-gene contact values between the `mm10_TSS.bed` file and the mESC DS011 `peak_df.parquet` file. Let's look at the results:

In [ ]:
import os
import scipy.sparse
import pandas as pd
import numpy as np

project_dir = "/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/"

# Input files
peak_location_file = os.path.join(project_dir, "output/DS011_mESC/DS011_mESC_sample1/tmp/peak_df.parquet")
gene_tss_bedfile = os.path.join(project_dir, "data/genome_annotation/mm10/mm10_TSS.bed")

# Output file
contact_matrix_file = os.path.join(project_dir, 'dev/notebooks/hic_peak_to_tg_ds011_contact_matrix_peak_x_gene.npz')

sparse_contact_matrix  = scipy.sparse.load_npz(contact_matrix_file)


Let's annotate the matrix with the genes and peaks. First we need to load in the genes from the gene TSS location bed file and format the peak positions to chr:start-end format.

In [ ]:
mm10_tss = pd.read_csv(
    gene_tss_bedfile, 
    sep="\t", 
    header=None, 
    index_col=None,
    names=["chrom", "start", "end", "name", "score", "strand"]
    )

atac_peaks = pd.read_parquet(peak_location_file)
atac_peaks = atac_peaks.rename(columns={"chr":"chrom"})
atac_peaks['chrom'] = 'chr' + atac_peaks['chrom'].astype(str)


atac_peaks["peak_id"] = [
    f"{c}:{s}-{e}" for c, s, e in zip(atac_peaks["chrom"], atac_peaks["start"], atac_peaks["end"])
]



Next, we will extract the gene names and formatted peak genomic coordinates.

In [ ]:
gene_positions = mm10_tss["name"].values
peak_positions = atac_peaks["peak_id"].values

We will create a melted DataFrame of peak-gene contacts with scores above 0.

In [ ]:
# Extract non-zero coordinates directly from sparse matrix
coo = sparse_contact_matrix.tocoo()

# Build DataFrame directly from sparse matrix coordinates
contact_df = pd.DataFrame({
    "peak_id": peak_positions[coo.row],
    "target_id": gene_positions[coo.col],
    "contact_value": coo.data
})
contact_df

In [ ]:
hic_peak_to_gene_file = os.path.join(project_dir, 'dev/notebooks/hic_peak_to_tg.parquet')

contact_df.to_parquet(hic_peak_to_gene_file)

In [ ]:
n_peaks = contact_df["peak_id"].nunique()
n_targets = contact_df["target_id"].nunique()
n_edges = len(contact_df)

print(f"Number of Peaks: {n_peaks:,}")
print(f"Number of Targets: {n_targets:,}")
print(f"Number of Edges: {n_edges:,}")

In [ ]:
from grn_inference.normalization import minmax_normalize_pandas

hic_norm_df = minmax_normalize_pandas(contact_df, ["contact_value"])
hic_norm_df

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,5))
plt.hist(hic_norm_df["contact_value"], bins=150, log=True)
plt.title(" Hi-C peak to gene TSS Contact Values")
plt.ylabel("Log10 Frequency of Contact Values")
plt.xlabel("Contact Value")
plt.show()

## Testing Hi-C Validation of Cicero's peak-TG Regualtory Potential Predictions

In [ ]:
cicero_df = pd.read_parquet("/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/output/DS011_mESC/DS011_mESC_sample1/cicero_peak_to_tg_scores.parquet", engine="pyarrow")
cicero_df

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,5))
plt.hist(cicero_df["cicero_score"], bins=150)
plt.title("Cicero Score Distributin")
plt.ylabel("Frequency")
plt.xlabel("Cicero Score")
plt.show()

In [ ]:
cicero_hic_merged = pd.merge(cicero_df, hic_norm_df, on=["peak_id", "target_id"], how="inner")
cicero_hic_merged

Determining how well the Cicero scores correlate with the contact value

In [ ]:
from sklearn.metrics import roc_auc_score

cicero_hic_merged = cicero_hic_merged.dropna(subset=["contact_value", "cicero_score"])

threshold = cicero_hic_merged["contact_value"].quantile(0.95)  # top 5% Hi-C contacts
cicero_hic_merged["label"] = (cicero_hic_merged["contact_value"] >= threshold).astype(int)

auroc = roc_auc_score(cicero_hic_merged["label"], cicero_hic_merged["cicero_score"])
print("AUROC:", auroc)


In [ ]:
import matplotlib.pyplot as plt

plt.hist2d(cicero_hic_merged["cicero_score"], cicero_hic_merged["contact_value"], bins=100, norm='log')
plt.xlabel("Cicero Score")
plt.ylabel("Hi-C Contact Value")
plt.title("Cicero vs. Hi-C (Joint Distribution)")
plt.colorbar(label="log10 Count")
plt.show()

## Testing Hi-C Validation of MIRA's peak-TG Regulatory Potential Predictions

MIRA attempts to model both local (LITE) and long-range (NITE) peak-TG regulatory interactions.

In [ ]:
def print_dataset_size(dataset_name, df):
    print(dataset_name)

    n_peaks = df["peak_id"].nunique()
    n_targets = df["target_id"].nunique()
    n_edges = len(df)

    print(f"  - Peaks: {n_peaks:,}")
    print(f"  - TGs: {n_targets:,}")
    print(f"  - Edges: {n_edges:,}")

In [ ]:
mira_peak_to_tg_df = pd.read_parquet("/gpfs/Home/esm5360/MIRA/mira-datasets/mESC_filtered_L2_E7.5_rep1/mESC_E7.5_rep1_mira_peak_to_tg_scores_nonzero.parquet", engine="pyarrow")
mira_peak_to_tg_df

In [ ]:
print_dataset_size("MIRA Peak-TG Regulatory Potential Scores", mira_peak_to_tg_df)

In [ ]:
peak_location_file = "/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/output/mESC/filtered_L2_E7.5_rep1/tmp/peak_df.parquet"

mm10_tss = pd.read_csv(
    gene_tss_bedfile, 
    sep="\t", 
    header=None, 
    index_col=None,
    names=["chrom", "start", "end", "name", "score", "strand"]
    )

atac_peaks = pd.read_parquet(peak_location_file)
atac_peaks = atac_peaks.rename(columns={"chr":"chrom"})
atac_peaks['chrom'] = 'chr' + atac_peaks['chrom'].astype(str)


atac_peaks["peak_id"] = [
    f"{c}:{s}-{e}" for c, s, e in zip(atac_peaks["chrom"], atac_peaks["start"], atac_peaks["end"])
]

gene_positions = mm10_tss["name"].values
peak_positions = atac_peaks["peak_id"].values

# Extract non-zero coordinates directly from sparse matrix
coo = sparse_contact_matrix.tocoo()

# Build DataFrame directly from sparse matrix coordinates
contact_df = pd.DataFrame({
    "peak_id": peak_positions[coo.row],
    "target_id": gene_positions[coo.col],
    "contact_value": coo.data
})

from grn_inference.normalization import minmax_normalize_pandas

# hic_norm_df = minmax_normalize_pandas(contact_df, ["contact_value"])

print_dataset_size("Hi-C Contact Values for Peaks Within 1 Mb of a TG for mESC filtered L2 E7.5 rep1", contact_df)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,5))
plt.hist(mira_peak_to_tg_df["LITE_score"], bins=50, log=True)
plt.title("Distribution of MIRA LITE Scores")
plt.ylabel("Log10 Frequency of Scores")
plt.xlabel("MIRA LITE Score")
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.hist(mira_peak_to_tg_df["NITE_score"], bins=50, log=True)
plt.title("Distribution of MIRA NITE Scores")
plt.ylabel("Log10 Frequency of Scores")
plt.xlabel("MIRA NITE Score")
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.hist(mira_peak_to_tg_df["avg_chromatin_differential"], bins=50, log=True)
plt.title("Distribution of MIRA Average Chromatin Differential Scores")
plt.ylabel("Log10 Frequency of Scores")
plt.xlabel("MIRA Average Chromatin Differential Score")
plt.show()

In [ ]:
mira_hic_total_merge = pd.merge(mira_peak_to_tg_df, contact_df, on=["peak_id", "target_id"], how="outer", indicator=True)
both_mira_and_hic = mira_hic_total_merge[mira_hic_total_merge["_merge"] == "both"].drop(columns="_merge")
mira_not_in_hic = mira_hic_total_merge[mira_hic_total_merge["_merge"] == "left_only"].drop(columns="_merge")
hic_not_in_mira = mira_hic_total_merge[mira_hic_total_merge["_merge"] == "right_only"].drop(columns="_merge")

print_dataset_size("Peak-TG Edges in Both MIRA and in the Hi-C Contact Matrix", both_mira_and_hic)
print_dataset_size("Peak-TG Edges in MIRA but not in the Hi-C Contact Matrix", mira_not_in_hic)
print_dataset_size("Peak-TG Edges not in MIRA but in the Hi-C Contact Matrix", hic_not_in_mira)

In [ ]:
both_mira_and_hic

In [ ]:
print(f"Average LITE Score for edges NOT in Hi-C: {mira_not_in_hic['LITE_score'].mean()}")
print(f"Average LITE Score for edges IN Hi-C: {both_mira_and_hic['LITE_score'].mean()}")

print(f"Average NITE Score for edges NOT in Hi-C: {mira_not_in_hic['NITE_score'].mean()}")
print(f"Average NITE Score for edges IN Hi-C: {both_mira_and_hic['NITE_score'].mean()}")

print(f"Average Chromatin Differential Score for edges NOT in Hi-C: {mira_not_in_hic['avg_chromatin_differential'].mean()}")
print(f"Average Chromatin Differential Score for edges IN Hi-C: {both_mira_and_hic['avg_chromatin_differential'].mean()}")

In [ ]:
balanced_both_mira_and_hic = both_mira_and_hic.sample(min(len(both_mira_and_hic), len(mira_not_in_hic)))
balanced_mira_not_in_hic = mira_not_in_hic.sample(min(len(both_mira_and_hic), len(mira_not_in_hic)))

In [ ]:
from scipy.stats import pearsonr

corr_lite = pearsonr(balanced_both_mira_and_hic["LITE_score"], balanced_mira_not_in_hic["LITE_score"])
corr_nite = pearsonr(balanced_both_mira_and_hic["NITE_score"], balanced_mira_not_in_hic["NITE_score"])
corr_chrom = pearsonr(balanced_both_mira_and_hic["avg_chromatin_differential"], balanced_mira_not_in_hic["avg_chromatin_differential"])

print("LITE Pearson Correlation:")
print(f"  - Statistic: {corr_lite[0]:.4f}")
print(f"  - p-value: {corr_lite[1]:.2f}")

print("\nNITE Pearson Correlation:")
print(f"  - Statistic: {corr_nite[0]:.4f}")
print(f"  - p-value: {corr_nite[1]:.2f}")

print("\nChromatin Differential Pearson Correlation:")
print(f"  - Statistic: {corr_chrom[0]:.4f}")
print(f"  - p-value: {corr_chrom[1]:.2f}")

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(
    balanced_both_mira_and_hic["LITE_score"],
    bins=50,
    alpha=0.5,
    color="#4195df",
    # label="Edge in both MIRA and Hi-C",
    log=True
)
plt.hist(
    balanced_mira_not_in_hic["LITE_score"],
    bins=50,
    alpha=0.5,
    color="#747474",
    # label="Edge Only in MIRA",
    log=True
)
# plt.title("MIRA LITE Scores for Peak-TG Edges in Both MIRA and Hi-C vs Edges Only in MIRA", fontsize=16)
plt.xlabel("MIRA LITE Score", fontsize=18)
plt.ylabel("log10 Frequency", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(
    balanced_both_mira_and_hic["NITE_score"],
    bins=50,
    alpha=0.5,
    color="#4195df",
    # label="Edge in both MIRA and Hi-C",
    log=True
)
plt.hist(
    balanced_mira_not_in_hic["NITE_score"],
    bins=50,
    alpha=0.5,
    color="#747474",
    # label="Edge Only in MIRA",
    log=True
)
# plt.title("MIRA NITE Scores for Peak-TG Edges in Both MIRA and Hi-C vs Edges Only in MIRA", fontsize=16)
plt.xlabel("MIRA NITE Score", fontsize=18)
plt.ylabel("log10 Frequency", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(
    balanced_both_mira_and_hic["avg_chromatin_differential"],
    bins=50,
    alpha=0.5,
    color="#4195df",
    # label="Edge in both MIRA and Hi-C",
    log=True
)
plt.hist(
    balanced_mira_not_in_hic["avg_chromatin_differential"],
    bins=50,
    alpha=0.5,
    color="#747474",
    # label="Edge Only in MIRA",
    log=True
)
# plt.title("MIRA Chromatin Differential Scores for Peak-TG Edges in Both MIRA and Hi-C vs Edges Only in MIRA", fontsize=16)
plt.xlabel("MIRA Chromatin Differential Score", fontsize=18)
plt.ylabel("log10 Frequency", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from scipy.stats import pearsonr

# Make sure both series are the same length and aligned
x_lite = balanced_both_mira_and_hic["LITE_score"].reset_index(drop=True)
x_nite = balanced_both_mira_and_hic["NITE_score"].reset_index(drop=True)
x_chrom = balanced_both_mira_and_hic["avg_chromatin_differential"].reset_index(drop=True)
y_contact = balanced_both_mira_and_hic["contact_value"].reset_index(drop=True)

# Combine and drop any rows with NaN or inf
def clean(x, y):
    df = pd.DataFrame({"x": x, "y": y})
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    return df["x"], df["y"]

x_lite_clean, y_contact_clean1 = clean(x_lite, y_contact)
x_nite_clean, y_contact_clean2 = clean(x_nite, y_contact)
x_chrom_clean, y_contact_clean3 = clean(x_chrom, y_contact)

# Correlations
corr_lite = pearsonr(x_lite_clean, y_contact_clean1)
corr_nite = pearsonr(x_nite_clean, y_contact_clean2)
corr_chrom = pearsonr(x_chrom_clean, y_contact_clean3)

# Print results
print("MIRA LITE score vs Hi-C contact value Pearson Correlation:")
print(f"  - Statistic: {corr_lite[0]:.4f}")
print(f"  - p-value: {corr_lite[1]:.2f}")

print("\nMIRA NITE score vs Hi-C contact value Pearson Correlation:")
print(f"  - Statistic: {corr_nite[0]:.4f}")
print(f"  - p-value: {corr_nite[1]:.2f}")

print("\nMIRA Chromatin Differential score vs Hi-C contact value Pearson Correlation:")
print(f"  - Statistic: {corr_chrom[0]:.4f}")
print(f"  - p-value: {corr_chrom[1]:.2f}")


In [ ]:
import seaborn as sns
plt.figure(figsize=(6,5))

sns.scatterplot(x="LITE_score", y="contact_value", data=both_mira_and_hic, alpha=0.3, s=100)
plt.xlabel("MIRA LITE Score", fontsize=18)
plt.ylabel("Hi-C Contact Value", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,5))

sns.scatterplot(x="NITE_score", y="contact_value", data=both_mira_and_hic, alpha=0.3, s=100)
plt.xlabel("MIRA NITE Score", fontsize=18)
plt.ylabel("Hi-C Contact Value", fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.scatterplot(x="avg_chromatin_differential", y="contact_value", data=both_mira_and_hic, alpha=0.3, s=100)
plt.xlabel("MIRA Chromatin Differential Score", fontsize=18)
plt.ylabel("Hi-C Contact Value", fontsize=18)
plt.xticks(np.arange(0, 0.5, step=0.1), fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.show()

---

### Citations
1. Yao, Lijing, et al. “Demystifying the Secret Mission of Enhancers: Linking Distal Regulatory Elements to Target Genes.” Critical Reviews in Biochemistry and Molecular Biology, vol. 50, no. 6, Nov. 2015, pp. 550–73. DOI.org (Crossref), https://doi.org/10.3109/10409238.2015.1087961.

2. Sanyal, Amartya, et al. “The Long-Range Interaction Landscape of Gene Promoters.” Nature, vol. 489, no. 7414, Sep. 2012, pp. 109–13. DOI.org (Crossref), https://doi.org/10.1038/nature11279.

3. Li, Guoliang, et al. “Extensive Promoter-Centered Chromatin Interactions Provide a Topological Basis for Transcription Regulation.” Cell, vol. 148, no. 1–2, Jan. 2012, pp. 84–98. DOI.org (Crossref), https://doi.org/10.1016/j.cell.2011.12.014.

   
4. Yao, Lijing, et al. “Inferring Regulatory Element Landscapes and Transcription Factor Networks from Cancer Methylomes.” Genome Biology, vol. 16, no. 1, May 2015, p. 105. DOI.org (Crossref), https://doi.org/10.1186/s13059-015-0668-3.


5. Jin, Fulai, et al. “A High-Resolution Map of the Three-Dimensional Chromatin Interactome in Human Cells.” Nature, vol. 503, no. 7475, Nov. 2013, pp. 290–94. DOI.org (Crossref), https://doi.org/10.1038/nature12644.

6. Mifsud, Borbala, et al. “Mapping Long-Range Promoter Contacts in Human Cells with High-Resolution Capture Hi-C.” Nature Genetics, vol. 47, no. 6, Jun. 2015, pp. 598–606. DOI.org (Crossref), https://doi.org/10.1038/ng.3286.

7. Mumbach, Maxwell R., et al. “Enhancer Connectome in Primary Human Cells Identifies Target Genes of Disease-Associated DNA Elements.” Nature Genetics, vol. 49, no. 11, Nov. 2017, pp. 1602–12. DOI.org (Crossref), https://doi.org/10.1038/ng.3963.

8. Javierre, Biola M., et al. “Lineage-Specific Genome Architecture Links Enhancers and Non-Coding Disease Variants to Target Gene Promoters.” Cell, vol. 167, no. 5, Nov. 2016, pp. 1369-1384.e19. DOI.org (Crossref), https://doi.org/10.1016/j.cell.2016.09.037.

9. Schoenfelder, Stefan, et al. “The Pluripotent Regulatory Circuitry Connecting Promoters to Their Long-Range Interacting Elements.” Genome Research, vol. 25, no. 4, Apr. 2015, pp. 582–97. DOI.org (Crossref), https://doi.org/10.1101/gr.185272.114.

10. Lettice, L. A. “A Long-Range Shh Enhancer Regulates Expression in the Developing Limb and Fin and Is Associated with Preaxial Polydactyly.” Human Molecular Genetics, vol. 12, no. 14, Jul. 2003, pp. 1725–35. DOI.org (Crossref), https://doi.org/10.1093/hmg/ddg180.

11. Pennacchio, Len A., et al. “Enhancers: Five Essential Questions.” Nature Reviews Genetics, vol. 14, no. 4, Apr. 2013, pp. 288–95. DOI.org (Crossref), https://doi.org/10.1038/nrg3458.

12. Kvon, Evgeny Z., et al. “Genome-Scale Functional Characterization of Drosophila Developmental Enhancers in Vivo.” Nature, vol. 512, no. 7512, Aug. 2014, pp. 91–95. DOI.org (Crossref), https://doi.org/10.1038/nature13395.

13. Narita, Takeo, and Chunaram Choudhary. Mammalian Enhancers and GWASs Act Proximally and Seldom Skip Active Genes. Genomics, 3 Dec. 2024. DOI.org (Crossref), https://doi.org/10.1101/2024.11.29.625864.

14. Zhang, Yubo, et al. “Chromatin Connectivity Maps Reveal Dynamic Promoter–Enhancer Long-Range Associations.” Nature, vol. 504, no. 7479, Dec. 2013, pp. 306–10. DOI.org (Crossref), https://doi.org/10.1038/nature12716.